in order to use this code, an array must be created that creates the number of basis functions and electrons on each atom
This is not automized yet
Otherwise, getting Mulliken charges works

In [1]:
first_row = ["H","He"]
second_row_s = ["Li","Be"]
second_row_p = ["B","C","N","O","F","Ne"]
third_row_s = ["Na","Mg"]
third_row_p = ["Al","Si","P","S","Cl","Ar"]

element_names = []
for element in first_row:
    element_names.append(element)
for element in second_row_s:
    element_names.append(element)
for element in second_row_p:
    element_names.append(element)
for element in third_row_s:
    element_names.append(element)
for element in third_row_p:
    element_names.append(element)


# assign number of electrons
n_electrons = {}
atomic_no = 1
for element in element_names:
    n_electrons[element] = atomic_no
    atomic_no += 1

print(n_electrons)


# assign number of basis functions per atom
nbf_sto3g = {}
for element in element_names:
    if element in first_row:
        nbf_sto3g[element] = 1
    elif element in second_row_s:
        nbf_sto3g[element] = 2
    elif element in second_row_p:
        nbf_sto3g[element] = 5
    elif element in third_row_s:
        nbf_sto3g[element] = 6
    elif element in third_row_p:
        nbf_sto3g[element] = 9

print(nbf_sto3g)




{'H': 1, 'He': 2, 'Li': 3, 'Be': 4, 'B': 5, 'C': 6, 'N': 7, 'O': 8, 'F': 9, 'Ne': 10, 'Na': 11, 'Mg': 12, 'Al': 13, 'Si': 14, 'P': 15, 'S': 16, 'Cl': 17, 'Ar': 18}
{'H': 1, 'He': 1, 'Li': 2, 'Be': 2, 'B': 5, 'C': 5, 'N': 5, 'O': 5, 'F': 5, 'Ne': 5, 'Na': 6, 'Mg': 6, 'Al': 9, 'Si': 9, 'P': 9, 'S': 9, 'Cl': 9, 'Ar': 9}


In [2]:
#### The Mulliken Charges are also part of the molden file, so let us calculate them:
def get_density_mat(C,n_occ):
    nbf = C.shape[0]
    D = np.zeros((nbf,nbf))
    for mu in range(nbf):
        for nu in range(nbf):
            for i in range(n_occ):
                D[mu,nu] += 2*C[mu,i]*C[nu,i]
    return D

def get_Mulliken_charge(C,S,n_occ,nbf_on_A, nel_on_A):
    #calculate density matrix
    D = get_density_mat(C=C,n_occ=n_occ)

    #calculate population matrix (elementwise multiplication of D and S)
    #population = np.zeros((nbf,nbf))

    #Pmunu = Dmunu * Smunu
    population = D * S

    #get gross orbital product for orbital mu (GOP_mu) by summing over nu
    nbf = population.shape[0]
    totalGOP=0 #=N_elec
    GOP_mus = []
    for mu in range(nbf):
        GOP_mu = 0
        for nu in range(nbf):
            GOP_mu += population[mu,nu]
        totalGOP += GOP_mu
        GOP_mus.append(GOP_mu)

    GOP_mus = np.asarray(GOP_mus)

    index = 0
    GOP_A = np.zeros(len(nbf_on_A))
    for atom in range(len(nbf_on_A)):
        #print(index,nbf_on_A[atom]+nbf_on_A[atom])
        for mu in range(index,index+nbf_on_A[atom]):
            GOP_A[atom] += GOP_mus[mu]
        index += nbf_on_A[atom]

    #print('GOP_mus =           ',GOP_mus)
    #print('GOP_A =             ',GOP_A)
    #print('total GOP = N_elec =',totalGOP)
    return nel_on_A - GOP_A

In [3]:
with open("propane.xyz") as file:
    lines = ""
    for line in file:
        lines += line

print(lines)

11
generated from 'CCC' smiles using OpenBabel
C          1.04355        0.06827        0.07733
C          2.56246        0.04540        0.06162
C          3.09472       -1.13362       -0.73524
H          0.68203        0.92426        0.65567
H          0.64232       -0.84319        0.53189
H          0.64265        0.15047       -0.93819
H          2.93766       -0.00935        1.08979
H          2.93744        0.97873       -0.37267
H          2.75620       -2.08184       -0.30545
H          4.18909       -1.13083       -0.73320
H          2.75646       -1.08803       -1.77536




In [4]:
import numpy as np
import psi4
np.set_printoptions(precision=4,suppress=True,linewidth=200)

#openmolcas, mulliken charge for propane: -0.1711 -0.0906 -0.1711  0.0550  0.0546  0.0546  0.0522  0.0523  0.0546  0.0550  0.0546
#openmolcas, HF/sto3g energy: -116.885122921


psi4.core.set_output_file('output.dat',False)

moldoc = """
11
generated from 'CCC' smiles using OpenBabel
C          1.04355        0.06827        0.07733
C          2.56246        0.04540        0.06162
C          3.09472       -1.13362       -0.73524
H          0.68203        0.92426        0.65567
H          0.64232       -0.84319        0.53189
H          0.64265        0.15047       -0.93819
H          2.93766       -0.00935        1.08979
H          2.93744        0.97873       -0.37267
H          2.75620       -2.08184       -0.30545
H          4.18909       -1.13083       -0.73320
H          2.75646       -1.08803       -1.77536"""


mol=psi4.geometry(lines)
mol.update_geometry()


# DEFINE nbf_on_A and nel_on_A
#assign for xyz
atom_list = []
for line in moldoc.split("\n"):
    for element in element_names:
        if line.startswith(element):
            atom_list.append(element)
print(atom_list)

nbf_on_A = []
for atom in atom_list:
    nbf_on_A.append(nbf_sto3g[atom])
nbf_on_A = np.asarray(nbf_on_A)

nel_on_A = []
for atom in atom_list:
    nel_on_A.append(n_electrons[atom])
nel_on_A = np.asarray(nel_on_A)

#nbf_on_A = np.array([5,5,5,1,1,1,1,1,1,1,1])
#nel_on_A = np.array([6,6,6,1,1,1,1,1,1,1,1])
print(nbf_on_A,nel_on_A)

['C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H']
[5 5 5 1 1 1 1 1 1 1 1] [6 6 6 1 1 1 1 1 1 1 1]


In [5]:

#specification of a basis set and additional options
psi4.set_options({'basis':        'sto-3g',
                  'scf_type':         'pk',
                  'e_convergence':    1e-8,
                  'd_convergence':    1e-8})

#Hartree-Fock calculation for the water molecule
energy,wfn=psi4.energy('hf',molecule=mol,return_wfn=True)

mints = psi4.core.MintsHelper(wfn.basisset())

print("HF energy of the water molecule:",energy)
CMO = np.asarray(wfn.Ca())
S = np.asarray(mints.ao_overlap())
n_occ = wfn.nalpha()


if n_occ != sum(nel_on_A)/2:
    print("WARNING: check no of electrons, sth doesnt add up")
if S.shape[0] != sum(nbf_on_A):
    print("WARNING: check no of basis functions, sth doesnt add up")

HF energy of the water molecule: -116.88512292398215


In [6]:
get_Mulliken_charge(CMO,S,n_occ,nbf_on_A, nel_on_A)
#openmolcas, mulliken charge for propane: -0.1711 -0.0906 -0.1711  0.0550  0.0546  0.0546  0.0522  0.0523  0.0546  0.0550  0.0546


array([-0.1711, -0.0906, -0.1711,  0.055 ,  0.0546,  0.0546,  0.0522,  0.0523,  0.0546,  0.055 ,  0.0546])